In [ ]:
from matplotlib.dates import relativedelta
from regex import F
from assetAllocation import AssetAllocation
from commonHelper import EDateType
import commonHelper
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from portfolio import Portfolio
import yfinance as yf
import pandas as pd
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

start_date = '2013-12-01'
end_date = '2025-08-26'
symbols = ['A', 'AAPL', 'AMD', 'TSLA']

df_quarter = DB_FinancialStatement.get_fs_data_static(symbols=symbols)
# display(df_quarter)
    

Portfolio.show_portfolio_eft()